# 03 - Generación de Reportes

Este notebook genera reportes de rendimiento para cada cliente.

**Proceso:**
1. Carga históricos de cuentas y precios
2. Calcula VCP (Valor Cuotaparte) - **FOTO**
3. Genera evolución temporal del VCP - **PELÍCULA**
4. Guarda reportes en `data/03_analytics/reports/`

In [ ]:
import sys
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

# Agregar src al path
sys.path.append(str(Path.cwd().parent / 'src'))

from core.account_manager import AccountManager
from core.pricing_engine import PricingEngine
from core.fci_calculator import FCICalculator
from utils.file_io import load_yaml_config

# Configurar visualización
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Configuración de rutas
BASE_PATH = Path.cwd().parent
CONFIG_PATH = BASE_PATH / 'config'
DATA_PATH = BASE_PATH / 'data'
PROCESSED_PATH = DATA_PATH / '02_processed'
ANALYTICS_PATH = DATA_PATH / '03_analytics'

# Cargar configuración
clients_config = load_yaml_config(CONFIG_PATH / 'clients_config.yaml')
clients = [c for c in clients_config['clients'] if c['active']]

# Inicializar componentes
account_mgr = AccountManager(PROCESSED_PATH)
pricing_engine = PricingEngine(PROCESSED_PATH, CONFIG_PATH)
fci_calc = FCICalculator(PROCESSED_PATH, ANALYTICS_PATH)

In [ ]:
# Cargar precios maestros
master_prices = pricing_engine.get_master_prices()

if master_prices is None or master_prices.empty:
    raise ValueError("No hay precios disponibles. Ejecutar notebook 02 primero.")

print(f"Precios cargados: {len(master_prices):,} registros")
print(f"Rango: {master_prices['date'].min()} a {master_prices['date'].max()}")

In [ ]:
# Procesar cada cliente
for client in clients:
    client_id = client['id']
    
    print(f"\n{'='*60}")
    print(f"Cliente: {client['name']} ({client_id})")
    print(f"{'='*60}")
    
    # Cargar histórico de cuenta
    account_history = account_mgr.get_historical_account(client_id)
    
    if account_history is None or account_history.empty:
        print(f"⚠️  No hay datos históricos para {client_id}")
        continue
    
    print(f"Transacciones: {len(account_history)}")
    
    # Calcular FOTO (VCP actual)
    foto = fci_calc.calculate_vcp(client_id, account_history, master_prices)
    
    print(f"\nFOTO (Estado Actual):")
    print(f"  Fecha: {foto['fecha']}")
    print(f"  Patrimonio Total: ${foto['patrimonio_total']:,.2f}")
    print(f"  Valor Cartera: ${foto['valor_cartera']:,.2f}")
    print(f"  Efectivo: ${foto['efectivo']:,.2f}")
    print(f"  Cuotapartes: {foto['cuotapartes']:,.2f}")
    print(f"  VCP: ${foto['vcp']:,.4f}")
    
    # Calcular PELÍCULA (Evolución temporal)
    start_date = account_history['fecha'].min()
    end_date = datetime.now()
    
    print(f"\nGenerando PELÍCULA ({start_date} a {end_date})...")
    
    pelicula = fci_calc.generate_pelicula(
        client_id, account_history, master_prices, 
        start_date, end_date, frequency='D'
    )
    
    # Guardar reporte
    fci_calc.save_report(client_id, pelicula)
    
    # Mostrar estadísticas
    print(f"\nEstadísticas de Rendimiento:")
    print(f"  Rendimiento Acumulado: {pelicula['rendimiento_acumulado'].iloc[-1]*100:.2f}%")
    print(f"  Mejor Día: {pelicula['rendimiento_diario'].max()*100:.2f}%")
    print(f"  Peor Día: {pelicula['rendimiento_diario'].min()*100:.2f}%")
    print(f"  Volatilidad (std): {pelicula['rendimiento_diario'].std()*100:.2f}%")

In [ ]:
# Visualizar evolución del VCP
for client in clients:
    client_id = client['id']
    report_file = ANALYTICS_PATH / 'reports' / f"{client_id}_rendimiento_mensual.csv"
    
    if not report_file.exists():
        continue
    
    df = pd.read_csv(report_file, parse_dates=['fecha'])
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    # Gráfico 1: VCP
    axes[0].plot(df['fecha'], df['vcp'], linewidth=2, color='#2E86AB')
    axes[0].set_title(f'Evolución VCP - {client["name"]}', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('VCP ($)', fontsize=12)
    axes[0].grid(True, alpha=0.3)
    
    # Gráfico 2: Rendimiento Acumulado
    axes[1].plot(df['fecha'], df['rendimiento_acumulado']*100, linewidth=2, color='#A23B72')
    axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
    axes[1].set_title('Rendimiento Acumulado (%)', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Rendimiento (%)', fontsize=12)
    axes[1].set_xlabel('Fecha', fontsize=12)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
print("\n✓ Generación de reportes completada")